In [1]:
import sys
import requests
import time

url = "https://health.api.nvidia.com/v1/biology/mit/diffdock"
header_auth = "Bearer $API_KEY_REQUIRED_IF_EXECUTING_OUTSIDE_NGC"

In [2]:
def _upload_asset(input):
    assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"

    headers = {
        "Authorization": header_auth,
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    s3_headers = {
        "x-amz-meta-nvcf-asset-description": "diffdock-file",
        "content-type": "text/plain",
    }

    payload = {
        "contentType": "text/plain",
        "description": "diffdock-file"
    }

    response = requests.post(
        assets_url, headers=headers, json=payload, timeout=30
    )

    response.raise_for_status()

    asset_url = response.json()["uploadUrl"]
    asset_id = response.json()["assetId"]

    response = requests.put(
        asset_url,
        data=input,
        headers=s3_headers,
        timeout=300,
    )

    response.raise_for_status()
    return asset_id

In [3]:
with open("protein.pdb", "r") as f:
  protein = f.read()
  protein_id = _upload_asset(protein)
with open("molecule.sdf", "r") as f:
  molecule = f.read()
  ligand_id = _upload_asset(molecule)

In [4]:
headers = {
    "Content-Type": "application/json",
    "NVCF-INPUT-ASSET-REFERENCES": ",".join([protein_id, ligand_id]),
    "Authorization": header_auth
}

r = requests.post(url, headers=headers, json={
    "ligand": ligand_id,
    "ligand_file_type": "sdf",
    "protein": protein_id,
    "num_poses": 20,
    "time_divisions": 20,
    "steps": 18,
    "save_trajectory": True,
    "is_staged": True
})

In [5]:
print(r, url, r.text)

<Response [200]> https://health.api.nvidia.com/v1/biology/mit/diffdock {"trajectory":["MODEL\nHETATM    1  C1  UNL     1      -5.786  -3.343  -2.706  1.00  0.00           C  \nHETATM    2  N1  UNL     1      -4.502  -3.634  -2.078  1.00  0.00           N  \nHETATM    3  N2  UNL     1      -3.828  -2.655  -1.329  1.00  0.00           N  \nHETATM    4  C2  UNL     1      -4.289  -2.120  -0.234  1.00  0.00           C  \nHETATM    5  N3  UNL     1      -5.501  -2.584   0.376  1.00  0.00           N  \nHETATM    6 BR1  UNL     1      -5.842  -4.432   0.641  1.00  0.00          BR  \nHETATM    7  N4  UNL     1      -3.578  -1.034   0.377  1.00  0.00           N  \nHETATM    8  N5  UNL     1      -2.311  -0.612  -0.077  1.00  0.00           N  \nHETATM    9  C3  UNL     1      -1.258  -1.475  -0.100  1.00  0.00           C  \nHETATM   10  O1  UNL     1      -1.392  -2.666   0.290  1.00  0.00           O  \nHETATM   11  N6  UNL     1      -0.051  -1.055  -0.534  1.00  0.00           N  \nHETA